# Práctica 2: Caso práctico con Mongo DB.

In [1]:
# Imports
import pymongo
import json

# **EJERCICIO 2**

# *Apartado A*

Completa los datos de conexión a la BBDD MongoDB.

### Consulta en Python

In [2]:
# Conexiones con MongoDB 
db_name = "practica_2_mongodb"
db_uri = "mongodb://localhost:27017"
db_client = pymongo.MongoClient(db_uri)
db = db_client[db_name]

# Nombres de colecciones
nombre_coleccion_usuarios = "usuarios"
nombre_coleccion_partidas = "partidas"

# Referencias a colecciones
coleccion_usuarios = db[nombre_coleccion_usuarios]
coleccion_partidas = db[nombre_coleccion_partidas]

print(f"✅ Conectado a la base de datos: {db_name}")

✅ Conectado a la base de datos: practica_2_mongodb


### Explicación

Para conectarme a MongoDB desde Python he usado pymongo. La URI "mongodb://localhost:27017" apunta a mi servidor local de MongoDB que tengo corriendo en el puerto 27017. He creado referencias a las dos colecciones que voy a usar (usuarios y partidas) para poder trabajar con ellas más fácilmente en el resto de apartados.

# *Apartado B*

Genera una función (“check_if_exist(db, nombre_coleccion))” que compruebe si las colecciones existen y, en caso de existir, las elimine. A su vez, si la colección se ha borrado, que escriba un mensaje por pantalla y, si no existe, también.   
Usa dicha función con nuestras dos colecciones.

### Consulta en Python

In [6]:
def check_if_exist(db, nombre_coleccion):
    if nombre_coleccion in db.list_collection_names():
        db[nombre_coleccion].drop()
        print(f"Colección '{nombre_coleccion}' eliminada correctamente")
    else:
        print(f" La colección '{nombre_coleccion}' no existía previamente")

# Comprobar si existen 
check_if_exist(db, nombre_coleccion_usuarios)
check_if_exist(db, nombre_coleccion_partidas)

 La colección 'usuarios' no existía previamente
 La colección 'partidas' no existía previamente


### Explicación

Esta función me permite empezar con las colecciones limpias cada vez que ejecuto el notebook. Primero comprueba si la colección existe usando list_collection_names(), y si existe la elimina con drop(). Esto es útil porque así evito duplicar datos si ejecuto el código varias veces mientras lo pruebo.

# *Apartado C*

Carga los ficheros de datos JSON proporcionados en sus colecciones correspondientes.  
Para ello, genera una función genérica (“load_data(coleccion, ruta_fichero_json))” y úsala.

### Consulta en Python

In [15]:
def load_data(coleccion, ruta_fichero_json):
    with open(ruta_fichero_json, 'r', encoding='utf-8') as file:
        data = json.load(file)
        resultado = coleccion.insert_many(data)
        print(f"✅ {len(resultado.inserted_ids)} documentos cargados en '{coleccion.name}'")

ruta_coleccion_usuarios = "usuarios.json"
ruta_coleccion_partidas = "partidas.json"

load_data(coleccion_usuarios, ruta_coleccion_usuarios)
load_data(coleccion_partidas, ruta_coleccion_partidas)

print(f"\n📊 Total usuarios: {coleccion_usuarios.count_documents({})}")
print(f"📊 Total partidas: {coleccion_partidas.count_documents({})}")

✅ 15 documentos cargados en 'usuarios'
✅ 45 documentos cargados en 'partidas'

📊 Total usuarios: 15
📊 Total partidas: 45


### Explicación

He creado una función genérica que lee archivos JSON y los carga en MongoDB. Uso insert_many() porque es más eficiente que insertar documento por documento. La función abre el archivo, carga todos los datos a la vez, y los inserta en la colección correspondiente. Al final compruebo que se hayan cargado todos los documentos correctamente.

# *Apartado D*

Obtén las puntuaciones más altas de cada jugador. ( + El resultado debe mostrar los
campos “username” y “maxScore” , con esos nombres, siendo username el valor de
“user_id” y “maxScore” la puntuación más alta de cada jugador)

### Consulta en Python

In [18]:
# PRUEBA - Verificar datos
print("=== VERIFICACIÓN DE DATOS ===\n")

# Contar documentos
print(f"Total partidas: {coleccion_partidas.count_documents({})}")
print(f"Total usuarios: {coleccion_usuarios.count_documents({})}")

# Ver una partida de ejemplo
print("\n=== EJEMPLO DE PARTIDA ===")
ejemplo = coleccion_partidas.find_one()
if ejemplo:
    print(ejemplo)
else:
    print(" NO HAY PARTIDAS")

# Intentar la agregación manualmente
print("\n=== PRUEBA DE AGREGACIÓN ===")
pipeline = [
    {"$group": {"_id": "$user_id", "max_score": {"$max": "$score"}}},
    {"$sort": {"max_score": -1}}
]
resultados = list(coleccion_partidas.aggregate(pipeline))
print(f"Resultados encontrados: {len(resultados)}")
if resultados:
    print("Primer resultado:", resultados[0])

=== VERIFICACIÓN DE DATOS ===

Total partidas: 45
Total usuarios: 15

=== EJEMPLO DE PARTIDA ===
{'_id': ObjectId('6914edd00d8e5f02109e4d97'), 'user_id': 'PixelNinja', 'score': 18, 'date': '2024-01-16T09:00:00Z', 'duration': 140, 'level': 3, 'steps': 52, 'final_position': {'x': 4, 'y': 3}}

=== PRUEBA DE AGREGACIÓN ===
Resultados encontrados: 15
Primer resultado: {'_id': 'SlitherSage', 'max_score': 35}


In [27]:
# Puntuaciones más altas por jugador
pipeline = [
    {"$group": {
        "_id": "$user_id",
        "maxScore": {"$max": "$score"}
    }},
    {"$sort": {"maxScore": -1}},
    {"$project": {
        "_id": 0,
        "username": "$_id",
        "maxScore": 1
    }}
]

resultados = list(coleccion_partidas.aggregate(pipeline))

print("🏆 Puntuaciones más altas por jugador:\n")
for resultado in resultados:
    print(f"   • {resultado['username']}: {resultado['maxScore']} puntos")

🏆 Puntuaciones más altas por jugador:

   • SlitherSage: 35 puntos
   • MambaMaverick: 33 puntos
   • SlitherExpert: 30 puntos
   • FangFury: 28 puntos
   • CoilCrusher: 27 puntos
   • ViperWarrior: 25 puntos
   • CobraKing: 23 puntos
   • SerpentMaster: 22 puntos
   • ScaleSurgeon: 22 puntos
   • ReptileRider: 20 puntos
   • PixelNinja: 18 puntos
   • PythonLord: 18 puntos
   • SlyShadow: 17 puntos
   • VenomVortex: 14 puntos
   • AnacondaAce: 12 puntos


### Explicación

Para obtener la puntuación más alta de cada jugador he usado el framework de agregación de MongoDB. Primero agrupo todas las partidas por user_id y uso el operador $max para quedarme solo con el score más alto de cada uno. Luego ordeno los resultados de mayor a menor puntuación para ver quiénes son los mejores jugadores.

# *Apartado E*

Lista todos los jugadores que hayan superado el nivel 3 y ordénalos de mayor a menor por nivel.

### Consulta en Python

In [20]:
# Jugadores que superaron el nivel 3
jugadores = coleccion_partidas.find(
    {"level": {"$gt": 3}},
    {"user_id": 1, "level": 1, "score": 1, "_id": 0}
).sort("level", -1)

print(" Jugadores que superaron el nivel 3:\n")
for jugador in jugadores:
    print(f"   • {jugador['user_id']} - Nivel {jugador['level']} - Score {jugador['score']}")


 Jugadores que superaron el nivel 3:

   • SlitherSage - Nivel 7 - Score 35
   • MambaMaverick - Nivel 6 - Score 33
   • SlitherExpert - Nivel 6 - Score 30
   • SlitherSage - Nivel 6 - Score 32
   • SlitherExpert - Nivel 6 - Score 30
   • FangFury - Nivel 5 - Score 28
   • SlitherSage - Nivel 5 - Score 25
   • FangFury - Nivel 5 - Score 26
   • CoilCrusher - Nivel 5 - Score 27
   • ViperWarrior - Nivel 5 - Score 25
   • FangFury - Nivel 5 - Score 27
   • MambaMaverick - Nivel 5 - Score 24
   • CobraKing - Nivel 4 - Score 23
   • CoilCrusher - Nivel 4 - Score 20
   • ScaleSurgeon - Nivel 4 - Score 22
   • CobraKing - Nivel 4 - Score 19
   • MambaMaverick - Nivel 4 - Score 21
   • ScaleSurgeon - Nivel 4 - Score 20
   • SerpentMaster - Nivel 4 - Score 22
   • PythonLord - Nivel 4 - Score 18
   • CoilCrusher - Nivel 4 - Score 21
   • ReptileRider - Nivel 4 - Score 20


### Explicación

Aquí filtro las partidas donde el nivel sea mayor que 3 usando el operador $gt (greater than). En la proyección selecciono solo los campos que me interesan (user_id, level y score) y excluyo el _id para que la salida sea más limpia. Finalmente ordeno por nivel de mayor a menor con sort("level", -1) para ver primero a los que llegaron más lejos.

# *Apartado F*

Busca todas las partidas jugadas entre el 01/03/2024 y el 30/04/2024. (en el enunciado pone otra fecha asi que la hare como lo pone en el enunciado  tambien 

### Consulta en Python

In [21]:
# Partidas entre 01/03/2024 y 30/04/2024
partidas = list(coleccion_partidas.find({
    "date": {
        "$gte": "2024-03-01T00:00:00Z",
        "$lte": "2024-04-30T23:59:59Z"
    }
}, {"user_id": 1, "date": 1, "score": 1, "_id": 0}))

print("📅 Partidas entre 01/03/2024 y 30/04/2024:\n")
for i, partida in enumerate(partidas, 1):
    print(f"   {i}. {partida['user_id']} - {partida['date'][:10]} - Score: {partida['score']}")

print(f"\n📊 Total: {len(partidas)} partidas")



📅 Partidas entre 01/03/2024 y 30/04/2024:

   1. SlyShadow - 2024-03-03 - Score: 14
   2. MambaMaverick - 2024-03-06 - Score: 33
   3. PythonLord - 2024-03-12 - Score: 11
   4. VenomVortex - 2024-03-15 - Score: 7
   5. CoilCrusher - 2024-03-20 - Score: 20
   6. SlitherSage - 2024-03-27 - Score: 25
   7. AnacondaAce - 2024-03-30 - Score: 9
   8. ReptileRider - 2024-04-07 - Score: 16
   9. ScaleSurgeon - 2024-04-12 - Score: 22
   10. SlyShadow - 2024-03-05 - Score: 17
   11. MambaMaverick - 2024-03-09 - Score: 21
   12. PythonLord - 2024-03-12 - Score: 15
   13. VenomVortex - 2024-03-19 - Score: 13
   14. CoilCrusher - 2024-03-24 - Score: 27
   15. SlitherSage - 2024-03-28 - Score: 32
   16. AnacondaAce - 2024-04-03 - Score: 10
   17. ReptileRider - 2024-04-06 - Score: 18
   18. ScaleSurgeon - 2024-04-10 - Score: 20
   19. SlyShadow - 2024-03-07 - Score: 16
   20. MambaMaverick - 2024-03-11 - Score: 24
   21. PythonLord - 2024-03-15 - Score: 18
   22. VenomVortex - 2024-03-20 - Score: 14

In [29]:
# Partidas entre 01/03/2024 y 31/03/2024
partidas = list(coleccion_partidas.find({
    "date": {
        "$gte": "2024-03-01T00:00:00Z",
        "$lte": "2024-03-31T23:59:59Z"
    }
}, {"user_id": 1, "date": 1, "score": 1, "_id": 0}))

print("📅 Partidas entre 01/03/2024 y 31/03/2024:\n")
for i, partida in enumerate(partidas, 1):
    print(f"   {i}. {partida['user_id']} - {partida['date'][:10]} - Score: {partida['score']}")

print(f"\n📊 Total: {len(partidas)} partidas")

📅 Partidas entre 01/03/2024 y 31/03/2024:

   1. SlyShadow - 2024-03-03 - Score: 14
   2. MambaMaverick - 2024-03-06 - Score: 33
   3. PythonLord - 2024-03-12 - Score: 11
   4. VenomVortex - 2024-03-15 - Score: 7
   5. CoilCrusher - 2024-03-20 - Score: 20
   6. SlitherSage - 2024-03-27 - Score: 25
   7. AnacondaAce - 2024-03-30 - Score: 9
   8. SlyShadow - 2024-03-05 - Score: 17
   9. MambaMaverick - 2024-03-09 - Score: 21
   10. PythonLord - 2024-03-12 - Score: 15
   11. VenomVortex - 2024-03-19 - Score: 13
   12. CoilCrusher - 2024-03-24 - Score: 27
   13. SlitherSage - 2024-03-28 - Score: 32
   14. SlyShadow - 2024-03-07 - Score: 16
   15. MambaMaverick - 2024-03-11 - Score: 24
   16. PythonLord - 2024-03-15 - Score: 18
   17. VenomVortex - 2024-03-20 - Score: 14
   18. CoilCrusher - 2024-03-25 - Score: 21
   19. SlitherSage - 2024-03-28 - Score: 35

📊 Total: 19 partidas


### Explicación

Para buscar partidas en un rango de fechas uso los operadores $gte (mayor o igual) y $lte (menor o igual) sobre el campo date. Las fechas están en formato ISO 8601 con la Z al final que indica UTC. He usado [:10] para mostrar solo la fecha sin la hora porque así queda más legible en la salida.

Explícación 2: Para buscar partidas en un rango de fechas uso los operadores $gte (mayor o igual) y $lte (menor o igual) sobre el campo date. En este caso filtro todas las partidas de marzo de 2024. Las fechas están en formato ISO 8601 con la Z al final que indica UTC.

# *Apartado G*

Ranking de jugadores por puntuación total acumulada.

### Consulta en Python

In [22]:
# Ranking por puntuación total acumulada
pipeline = [
    {"$group": {
        "_id": "$user_id",
        "total_score": {"$sum": "$score"},
        "partidas_jugadas": {"$sum": 1}
    }},
    {"$sort": {"total_score": -1}}
]

ranking = list(coleccion_partidas.aggregate(pipeline))

print("🏆 RANKING - Puntuación total acumulada:\n")
for posicion, jugador in enumerate(ranking, 1):
    print(f"   {posicion}. {jugador['_id']}: {jugador['total_score']} puntos ({jugador['partidas_jugadas']} partidas)")

🏆 RANKING - Puntuación total acumulada:

   1. SlitherSage: 92 puntos (3 partidas)
   2. FangFury: 81 puntos (3 partidas)
   3. MambaMaverick: 78 puntos (3 partidas)
   4. SlitherExpert: 72 puntos (3 partidas)
   5. CoilCrusher: 68 puntos (3 partidas)
   6. ScaleSurgeon: 59 puntos (3 partidas)
   7. CobraKing: 57 puntos (3 partidas)
   8. ReptileRider: 54 puntos (3 partidas)
   9. SlyShadow: 47 puntos (3 partidas)
   10. PythonLord: 44 puntos (3 partidas)
   11. ViperWarrior: 44 puntos (3 partidas)
   12. SerpentMaster: 42 puntos (3 partidas)
   13. PixelNinja: 37 puntos (3 partidas)
   14. VenomVortex: 34 puntos (3 partidas)
   15. AnacondaAce: 31 puntos (3 partidas)


### Explicación

Este ranking suma todos los puntos que ha conseguido cada jugador en todas sus partidas. Uso $sum sobre el campo score para acumular los puntos, y también cuento cuántas partidas ha jugado cada uno usando $sum: 1. Esto me da una visión más completa porque puedo ver si alguien tiene muchos puntos por jugar más veces o por ser realmente bueno.

# *Apartado H*

Encontrar el usuario con la partida más larga en duración

### Consulta en Python

In [31]:
# Usuario con la partida más larga en duración esta vez mostrando solo los campos user_id y duration.
partida_mas_larga = coleccion_partidas.find_one(
    {},
    {"user_id": 1, "duration": 1, "_id": 0},
    sort=[("duration", -1)]
)

print("⏱️ Usuario con la partida más larga:\n")
print(f"   Usuario: {partida_mas_larga['user_id']}")
print(f"   Duración: {partida_mas_larga['duration']:.2f} segundos ({partida_mas_larga['duration']/60:.2f} minutos)")

⏱️ Usuario con la partida más larga:

   Usuario: SlitherSage
   Duración: 260.00 segundos (4.33 minutos)


In [23]:
# Usuario con la partida más larga en duración
partida_mas_larga = coleccion_partidas.find_one(
    {},
    {"user_id": 1, "duration": 1, "score": 1, "level": 1, "_id": 0},
    sort=[("duration", -1)]
)

print("⏱️ Usuario con la partida más larga:\n")
print(f"   Usuario: {partida_mas_larga['user_id']}")
print(f"   Duración: {partida_mas_larga['duration']:.2f} segundos ({partida_mas_larga['duration']/60:.2f} minutos)")
print(f"   Score: {partida_mas_larga['score']}")
print(f"   Nivel: {partida_mas_larga['level']}")

⏱️ Usuario con la partida más larga:

   Usuario: SlitherSage
   Duración: 260.00 segundos (4.33 minutos)
   Score: 35
   Nivel: 7


### Explicación

He usado find_one() con sort para obtener directamente la partida con mayor duración. El parámetro sort=[("duration", -1)] ordena por duración de mayor a menor, y find_one() me devuelve solo el primer resultado, que es el que busco. He convertido los segundos a minutos también para que sea más fácil entender cuánto tiempo jugó.

# *Apartado I*

Queremos conocer el correo electrónico del jugador con la puntuación más alta.  
La salida debe contener solo el registro de dicho jugador, mostrando los campos username, email y score.

### Consulta en Python

In [25]:
# Email del jugador con la puntuación más alta
pipeline = [
    {"$sort": {"score": -1}},
    {"$limit": 1},
    {"$lookup": {
        "from": "usuarios",
        "localField": "user_id",
        "foreignField": "username",
        "as": "user_info"
    }},
    {"$unwind": "$user_info"},
    {"$project": {
        "_id": 0,
        "username": "$user_id",
        "email": "$user_info.email",
        "score": 1
    }}
]

resultado = list(coleccion_partidas.aggregate(pipeline))

if resultado:
    jugador = resultado[0]
    print("🥇 Jugador con la puntuación más alta (Olé por él):\n")
    print(f"   Username: {jugador['username']}")
    print(f"   Email: {jugador['email']}")
    print(f"   Score máximo: {jugador['score']} puntos")

🥇 Jugador con la puntuación más alta (Olé por él):

   Username: SlitherSage
   Email: slithersage@example.com
   Score máximo: 35 puntos


### Explicación

Este apartado es el más complejo porque necesito combinar datos de dos colecciones. Primero ordeno las partidas por score y me quedo con la mejor usando $limit: 1. Luego hago un $lookup (que es como un JOIN en SQL) para traer los datos del usuario desde la colección usuarios. El $unwind descompone el array resultante para tener un documento plano. Así puedo mostrar el email del mejor jugador junto con su puntuación récord.

## Reflexión final

Bueno, pues después de darle bastante a esta práctica puedo decir que MongoDB empieza a gustarme. Al principio de la asignatura me parecía un rollo no tener tablas ni nada estructurado, pero ahora veo las ventajas cuando trabajas con datos que no encajan bien en formato tabla.

Los apartados que más me han costado han sido el I (ese $lookup con $unwind me ha traído por la calle de la amargura) y el G, porque no me quedaba claro cómo sumar dentro de un $group. Pero al final, probando y equivocándome, he conseguido que todo funcione.

Lo mejor ha sido comprobar que puedes hacer análisis bastante potentes directamente en la base de datos. Me quedo con eso para futuros proyectos.

---
By:la increible de Malu 